## Model description

A spring pendulum is a physical system where a piece of mass is connected to a spring so that the resulting motion contains elmeentso f both a simple pendulum and a one-dimensional spring-mass system. The model is also called elastic pendulum or swinging spring, we refer to the wikipedia article [Elastic pendulum](https://en.wikipedia.org/wiki/Elastic_pendulum) for further reading.

Consider the planar spring-pendulum depicted in the figure below, consisting of a solid ball of mass $m$ and a spring of natural length $L$. The spring constant is $k$. The differential equations that govern the evolution [1] of the length $r(t)$ of the spring and the angle $\theta(t)$ are considered next.

A textbook application of Newton's 2nd law of motion in polar coordinates gives
$$
  \left\{ \begin{array}{lcl} m\cdot \ddot{r} & = & m\cdot r\cdot \dot{\theta}^2 + m\cdot g \cdot \cos(\theta) - k\cdot (r - L) \\ r^2 \cdot \ddot{\theta} & = & -2\cdot r\cdot \dot{r} \cdot \dot{\theta} - g\cdot r\cdot \sin(\theta) \end{array}\right.  
$$

which can be equivalently translated to the first-order ODE as below:

$$
    \left\{  \begin{array}{lcl}  \dot{r} & = & v_r \\  \dot{\theta} & = & v_\theta \\  \dot{v}_r & = & r\cdot v_\theta^2 + g\cdot \cos(\theta) - k\cdot (r - L) \\  \dot{v}_\theta & = & -\frac{(2\cdot v_r \cdot v_\theta + g\cdot \sin(\theta))}{r} \\  \end{array}  \right.
$$

---

[1] J. D. Meiss. *Differential Dynamical Systems*, Monographs on Mathematical Modeling and Computation, Book 14, SIAM publishers, 2007.

<img src="spring_pendulum.png" alt="Drawing" style="width: 400px;"/>

The numerical values for the model's constants (in their respective units) are given in the following table.

|Quantity|Value|
|-----|-------|
|g | 9.81|
|k| 2|
|L | 1|

In [19]:
using Reachability, TaylorIntegration, MathematicalSystems

In [36]:
@taylorize function spring_pendulum!(du, u, p, t)
    local g, k, L = 9.8, 2.0, 1.0
    du[1] = u[3]
    du[2] = u[4]
    du[3] = ((u[1] * (u[4]*u[4])) + g*cos(u[2])) - k*(u[1]-L)
    du[4] = -((2*u[3]*u[4]) + g*sin(u[2]))/(u[1])
    
    # change of variables: r <- r - L
    #du[3] = (((u[1]+L) * (u[4]*u[4])) + g*cos(u[2])) - k*(u[1])
    #du[4] = -((2*u[3]*u[4]) + g*sin(u[2]))/(u[1]+L)
    
    return du
end

## Reachability settings

The settings are taken from the [HyPro benchmarks](https://ths.rwth-aachen.de/research/projects/hypro/spring-pendulum/).

We consider the initial set  $r \in [1.19,1.21]$,  $\theta\in [0.49,0.51]$, $v_r = 0$ and  $v_{\theta} = 0$.

In [37]:
r₀ = Interval(1.19, 1.21)
θ₀ = Interval(0.49, 0.51)
vr₀ = Singleton([0.0])
vθ₀ = Singleton([0.0])

X₀ = r₀ × θ₀ × vr₀ × vθ₀;

In [38]:
S = BlackBoxContinuousSystem(spring_pendulum!, 4)
P = InitialValueProblem(S, X₀);

## Results

In [39]:
sol = solve(P, Options(:T=>5.0),
            op=TMJets(Options(:max_steps=>1_000, :orderT=>8, :orderQ=>2,:output_type=>Zonotope)));

BoundsError: BoundsError: attempt to access 1-element Array{HomogeneousPolynomial{Float64},1} at index [2]

In [18]:
using Plots, LaTeXStrings
gr()

Plots.GRBackend()

In [ ]:
plot(sol, xlab=L"x", ylab=L"y", lw=0.2, color=:lightblue, lab="Flowpipe")
plot!(X₀, color=:orange, lab="Xo")d